In [1]:
import os, argparse, pickle
import matplotlib.pyplot as plt
import torch
from glob import glob
from utils import *
import models

In [2]:
args = argparse.ArgumentParser()
args.add_argument('--gpus', type=str, default='0')
args.add_argument('--batch', type=int, default=64)
args.add_argument('--len', type=int, default=40)
args.add_argument('--b', type=int, default=40)
args.add_argument('--mode', type=str, default='sj_S')
args.add_argument('--model', type=str, default='CombineAutoencoder')
config = args.parse_known_args()[0]

In [3]:
SR = 8192
WINDOW_SIZE = 500 # us
data_length = config.len
BATCH_SIZE = config.batch
K, m = 8, 8
ls = 128

In [4]:
ABSpath = '/home/skuser/'
path = os.path.join(ABSpath, 'ai_model/pytorch/test_model')
data_path = os.path.join(ABSpath,'data')
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
device =torch.device('cpu')
accel_raw_data = pickle.load(open(os.path.join(data_path,'stationary_accel_data.pickle'),'rb'))
sound_raw_data = pickle.load(open(os.path.join(data_path,'stationary_sound_data.pickle'),'rb'))
transfer_f = np.array(pickle.load(open(os.path.join(data_path,'transfer_f.pickle'),'rb')))
transfer_f = torch.from_numpy(transfer_f).to(device)
transfer_f.requires_grad = False

accel_data = dataSplit(accel_raw_data, takebeforetime=config.b, data_length=data_length, expand=True)
sound_data = dataSplit(sound_raw_data, takebeforetime=config.b, data_length=data_length, expand=False)
name = 'CombineAutoencoder_sj_S_40_40_adam_0.001_decay0.7071067811865475/*'
modelsavepath = sorted(glob(os.path.join(path, 'model_save/'+name)), key=lambda x: float(x.split('/')[-1].split('_')[-1]))[0]
model = getattr(models, config.model)(accel_data.shape[1] * accel_data.shape[2], sound_data.shape[1] * sound_data.shape[2]).to(device)

In [5]:
model.load_state_dict(torch.load(modelsavepath)['model'])

<All keys matched successfully>

In [6]:
res = []
dataset = makeDataset(accel_data, torch.zeros((accel_data.size(0),1,1)), train=False)
dataset_generator = DataLoader(dataset, batch_size=1, shuffle=False)
res = torch.cat([conv_with_S(model(data.to(device).transpose(1,2)), transfer_f, device) for data, _ in dataset_generator])

RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 7.94 GiB total capacity; 6.84 GiB already allocated; 768.00 KiB free; 6.84 GiB reserved in total by PyTorch)